In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import cv2
import os
from tqdm import tqdm
from PIL import Image

In [0]:

!wget https://www.dropbox.com/s/pxcz2wdz04zxocq/CamVid.zip?dl=1 -O CamVid.zip
!unzip CamVid.zip

In [0]:
class SeparableConv2d(nn.Module):
    def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
        super(SeparableConv2d,self).__init__()

        self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
        self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)
    
    def forward(self,x):
        x = self.conv1(x)
        x = self.pointwise(x)
        return x


class ConvBlock(nn.Module):
  def __init__(self,in_channels,out_channels,side_branch = True):
    super(ConvBlock,self).__init__()
    
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.side_branch = side_branch
    
    self.relu = nn.ReLU()
    
    self.sep_conv1 = SeparableConv2d(in_channels = self.in_channels,
                          out_channels = self.out_channels,
                          kernel_size = 3,
                          padding = 1)
    self.sep_conv2 = SeparableConv2d(in_channels = self.out_channels,
                          out_channels = self.out_channels,
                          kernel_size = 3,
                          padding = 1)
    self.sep_conv3 = SeparableConv2d(in_channels = self.in_channels,
                          out_channels = self.out_channels,
                          kernel_size = 3,
                          padding = 1,
                          stride = 2)
    self.conv4 = nn.Conv2d(in_channels = self.in_channels,
                          out_channels = self.out_channels,
                          kernel_size = 1,
                          padding = 1,
                          stride = 2)
    self.batchnorm = nn.BatchNorm2d()
      
        
  def forward(self,x):
    main = self.sep_conv1(x)    
    main = self.batchnorm(main)
    main = self.relu(main)
    main = self.sep_conv2(x)    
    main = self.batchnorm(main)
    main = self.relu(main)
    main = self.sep_conv3(x)    
    main = self.batchnorm(main)
    main = self.relu(main)
    
    if side_branch:
      side = F.relu(self.conv4(x))
      
    try:
      return side + main
    
    except:
      return x + main
    

    
    
    
    

In [0]:
class ASSP(nn.Module):
  def __init__(self,in_channels,out_channels = 256):
    super(ASSP,self).__init__()
    
    self.bn = nn.BatchNorm2d(out_channels)
    self.relu = nn.ReLU()
    
    self.conv1 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 1,
                          padding = 1)
    self.conv2 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 3,
                          padding = 6,
                          dilation = 6)
    self.conv3 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 3,
                          padding = 12,
                          dilation = 12)
    self.conv4 = nn.Conv2d(in_channels = in_channels, 
                          out_channels = out_channels,
                          kernel_size = 3,
                          padding = 18,
                          dilation = 18)
    self.adapool = nn.AdaptiveAvgPool2d((1,1))
   
  
  def forward(self,x):
    x1 = self.conv1(x)
    x1 = self.bn(x1)
    x1 = self.relu(x1)
    x2 = self.conv2(x)
    x2 = self.bn(x2)
    x2 = self.relu(x2)
    x3 = self.conv3(x)
    x3 = self.bn(x3)
    x3 = self.relu(x3)
    x4 = self.conv4(x)
    x4 = self.bn(x4)
    x4 = self.relu(x4)
    x5 = self.adapool(x)
    x5 = self.conv1(x5)
    x5 = self.nb(x5)
    x5 = self.relu(x5)
    
    x6 = torch.cat((x1,x2,x3,x4,x5), dim = 1) #channels first
    x6 = self.conv1(x6)
    x6 = self.bn(x6)
    
    return self.relu(x6)
    
    
    
    
      

In [0]:
# Checking if there is any gpu available and pass the model to gpu or cpu
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
enet = enet.to(device)

In [0]:
def loader(training_path, segmented_path, batch_size, h=320, w=1000):
    filenames_t = os.listdir(training_path)
    total_files_t = len(filenames_t)
    
    filenames_s = os.listdir(segmented_path)
    total_files_s = len(filenames_s)
    
    assert(total_files_t == total_files_s)
    
    if str(batch_size).lower() == 'all':
        batch_size = total_files_s
    
    idx = 0
    while(1):
      # Choosing random indexes of images and labels
        batch_idxs = np.random.randint(0, total_files_s, batch_size)
            
        
        inputs = []
        labels = []
        
        for jj in batch_idxs:
          # Reading normalized photo
            img = plt.imread(training_path + filenames_t[jj])
          # Resizing using nearest neighbor method
            img = cv2.resize(img, (h, w), cv2.INTER_NEAREST)
            inputs.append(img)
          
          # Reading semantic image
            img = Image.open(segmented_path + filenames_s[jj])
            img = np.array(img)
          # Resizing using nearest neighbor method
            img = cv2.resize(img, (h, w), cv2.INTER_NEAREST)
            labels.append(img)
         
        inputs = np.stack(inputs, axis=2)
      # Changing image format to C x H x W
        inputs = torch.tensor(inputs).transpose(0, 2).transpose(1, 3)
        
        labels = torch.tensor(labels)
        
        yield inputs, labels